## Discounter cash flow

DCF --- это метод оценки компании, для получения т.н. fair value (справедливая стоимость), с учетом будущих денежных потоков.  

простая формула может быть записана так:

FCF = PCF(1+r)^t

где FCF --- future cash flow, PCF --- present cash flow, r --- процентная ставка, t --- длительность инвестиций (в годах)

Если мы хотим посмотреть в будущее, то надо перенести правую часть формулы в левую:

$PCF = \frac{FCF}{(1+r)^t} $

Если мы хотим посмотреть в будущее не на один год, а на несколько (обычно берется 5-10 лет), то формула превращается в следующее:

$Intrinsic Value = \sum^{T}_{t=1}\frac{FCF}{(1+r)^t}$

Чтобы компанию можно было бы оценить при помощи DCF-анализа, она должна удовлетворять одной из четырех критериев:

1) Компания не платит дивидендов;

2) Компания платит мало дивидендов. Т.е. платит гораздо меньше, чем могла бы;

3) Размер свободного денежного потока совпадает или одного порядка с рентабельностью компании (для этого можно посмотреть FCF/Net Income); 

4) Инвестор рассматривает компанию в перспективе

5) Компания имеет положительный денежный поток

Для примера рассмотрим Apple:
![Apple](images/apple_cashflow.png)

Для рассчета свободного денежного потока (FCF --- free cash flow) берется операционный денежный поток (OCF --- operational cash flow) *минус* капитальные затраты (CAPEX --- capital expenditure) *плюс* net borrowing. Net borrowing находится в financing cash flow. <- **это сложный способ**

**Обычно net borrowing не учитывается.**

и используется упрощенная формула: 

$FCF = OCF - CAPEX$

In [18]:
fcf_year1 = 77434-13313
fcf_year2 = 69391-10495
fcf_year3 = 80674-7309
fcf_year4 = 104038-11085

fcf = []
fcf.append(fcf_year1)
fcf.append(fcf_year2)
fcf.append(fcf_year3)
fcf.append(fcf_year4)

income1 = 59531
income2 = 55256
income3 = 57411
income4 = 94680

net_income = []
net_income.append(income1)
net_income.append(income2)
net_income.append(income3)
net_income.append(income4)

rev1 = 265595
rev2 = 260174
rev3 = 274515
rev4 = 365817

revenue = []
revenue.append(rev1)
revenue.append(rev2)
revenue.append(rev3)
revenue.append(rev4)

In [19]:
import numpy as np

np.sum(np.array(fcf)/np.array(net_income))/4

1.1006571232225535

Далее, нам надо подсчитать **growth rate** и **net income margin**

**growth rate** --- мы просто берем средний рост выручки за последние 3-5 лет, желательно быть более консервативным и брать минимальное положительное значение 

**net income margin** --- мы разделяем net income (ЧП) на revenue (выручку)

In [20]:
growth_rate = 1.03
net_income_margin = np.array(net_income)/np.array(revenue)
print(net_income_margin)

[0.22414202 0.21238095 0.20913611 0.25881793]


Зная эти значения мы можем экстраполировать текущие данные на будущие

In [21]:
t = 5
rev = revenue[-1]
cash_flow = fcf[-1]
for i in range(1,t+1,1):
    new_revenue = int(rev*(growth_rate)**i)
    new_income = int(new_revenue*0.21)
    new_fcf = int(cash_flow*1.10**i)
    print(f'revenue:{new_revenue} income:{new_income} fcf:{new_fcf}')
    revenue.append(new_revenue)
    net_income.append(new_income)
    fcf.append(new_fcf)

revenue:376791 income:79126 fcf:102248
revenue:388095 income:81499 fcf:112473
revenue:399738 income:83944 fcf:123720
revenue:411730 income:86463 fcf:136092
revenue:424082 income:89057 fcf:149701


Далее, нам нужно значение **required return** --- это требуемая доходность (ТД). Пусть, для примера ТД будет **9%** (как в среднем у snp500)

И используем формулу:
$V_0 = \frac{FCFE_d\times(1+g)}{r-g}$, где

g --- perpetual growth (2.5% хорошее значение)
r --- ТД

In [22]:
perpetual_growth = 0.025
required_return = 0.09

In [23]:
# shares outstanding
shares = 16406.397 # in millions

In [24]:
terminal_value = fcf[-1]*(1+perpetual_growth)/(required_return - perpetual_growth)

print(terminal_value)

2360669.615384615


Теперь нам надо дисконтировать полученное значение.

Для дисконтирование используется discount_factor

$df = (1+r)^t$, где

r - required return
t - время (в годах)

In [25]:
discount_factor = []
for i in range(1,t+1):
    discount_factor.append((1+required_return)**i)

In [26]:
print(discount_factor)

[1.09, 1.1881000000000002, 1.2950290000000002, 1.4115816100000005, 1.5386239549000005]


теперь дисконтируем каждый расчитанный денежный поток

In [36]:
j = 0
dfcf = []
for i in range(t-1, len(fcf)):    
    dfcf.append(fcf[i]/discount_factor[j])
    j += 1

In [39]:
today_value = terminal_value / discount_factor[-1] + np.array(dfcf).sum()
print(today_value)

2011985.9766268183


In [40]:
intrinsic_value = today_value / shares
print(intrinsic_value)

122.6342369154433


In [1]:
path_cashflow = r"./Cashflows.csv"
path_rate = r"./Yield curve.csv"

# Implementation with Pandas
import pandas as pd

# Read in the cash flows data and rate data as csv
cashflow_df = pd.read_csv(path_cashflow)
rate_df = pd.read_csv(path_rate)

# Calculate discount factor from the rates
rate_df["Discount factor"] = 1 / (1 + rate_df["Interest rate"])**rate_df["Year"]

# Join cash flows with rates
cf_with_rate_df = cashflow_df.merge(rate_df, on=["Currency", "Year"], how="left")

# Calculate present values
cf_with_rate_df["Present value"] = cf_with_rate_df["Cash flows"] * cf_with_rate_df["Discount factor"]

# Groupby product and check the profitability
cf_with_rate_df = cf_with_rate_df.groupby("Product")[["Present value"]].sum().reset_index()


In [2]:
cf_with_rate_df

,Product,Present value
0,Fire insurance,993.107122
1,Flood insurance,7094.239858


### Implementation with Koalas


In [ ]:
import databricks.koalas as ks

# Read in the cash flows data and rate data as csv
cashflow_df = ks.read_csv(path_cashflow)
rate_df = ks.read_csv(path_rate)

# Calculate discount factor from the rates
rate_df["Discount factor"] = 1 / (1 + rate_df["Interest rate"])**rate_df["Year"]

# Join cash flows with rates
cf_with_rate_df = cashflow_df.merge(rate_df, on=["Currency", "Year"], how="left")

# Calculate present values
cf_with_rate_df["Present value"] = cf_with_rate_df["Cash flows"] * cf_with_rate_df["Discount factor"]

# Groupby product and check the profitability
cf_with_rate_df = cf_with_rate_df.groupby("Product")[["Present value"]].sum().reset_index()


### Implementation with PySpark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as f

# Define Spark settings
builder = SparkSession.builder.appName("Discount_Cashflows")
spark = builder.getOrCreate()

# Read in the cash flows data and rate data as csv
cashflow_df = spark.read.csv(path_cashflow, header=True, inferSchema=True)
rate_df = spark.read.csv(path_rate, header=True, inferSchema=True)

# Calculate discount factor from the rates
rate_df = rate_df.withColumn("Discount factor", 1 / (1 + rate_df["Interest rate"])**rate_df["Year"])

# Join cash flows with rates
cf_with_rate_df = cashflow_df.join(f.broadcast(rate_df), on=["Currency", "Year"], how="left")

# Calculate present values
cf_with_rate_df = cf_with_rate_df.withColumn("Present value", f.col("Cash flows") * f.col("Discount factor"))

# Groupby product and check the profitability
cf_with_rate_df = cf_with_rate_df.groupBy("Product").agg(f.sum("Present value").alias("Present value"))
